# Independent Verification of $C_{1a}$ Upper Bounds

## Summary

This notebook loads saved step-function solutions from `best_solutions.json` and
independently verifies the claimed upper bound $\|f*f\|_\infty / \|f\|_1^2$ using
four independent computation methods:

| # | Method | Description |
|---|--------|-------------|
| 1 | **Exact breakpoint** | Evaluates $(f*f)(t)$ at all $O(P^2)$ breakpoints where the piecewise-linear autoconvolution can change slope. |
| 2 | **Direct numerical integration** | Computes $(f*f)(t) = \int f(x)f(t-x)\,dx$ by summing interval overlaps on a fine grid. |
| 3 | **FFT convolution** | High-resolution FFT on a fine grid as a sanity check. |
| 4 | **Discrete simplex objective** | $\max_k 2P \sum_{i+j=k} x_i x_j$ directly from simplex weights. |

A feasibility checker also verifies that each solution satisfies $f \geq 0$ and $\int f = 1$.

All helper functions are defined inline (no external imports beyond `numpy` and `json`).

In [ ]:
import json
import numpy as np

# Load saved solutions (same directory as this notebook)
with open('best_solutions.json', 'r') as f:
    all_solutions = json.load(f)

print(f"Loaded {len(all_solutions)} solutions")
for label in sorted(all_solutions.keys()):
    s = all_solutions[label]
    claimed_key = 'claimed_exact_peak' if 'claimed_exact_peak' in s else 'exact_peak'
    print(f"  {label}: P={s['P']}, claimed={s[claimed_key]:.6f}")

In [ ]:
# === Verification Method 1: Exact breakpoint evaluation ===

def verify_exact_breakpoint(edges, heights):
    """
    Compute the exact peak of (f*f)(t) for a step function f.

    f(x) = heights[i] for edges[i] <= x < edges[i+1], zero outside.

    (f*f)(t) = integral f(x) f(t-x) dx

    This is piecewise linear in t, with breakpoints at t = edges[i] + edges[j].
    The maximum must occur at one of these breakpoints.
    """
    edges = np.asarray(edges, dtype=np.float64)
    heights = np.asarray(heights, dtype=np.float64)
    N = len(heights)

    # All breakpoints
    bp = (edges[:, None] + edges[None, :]).ravel()
    bp = np.unique(bp)
    bp = bp[(bp >= 2 * edges[0]) & (bp <= 2 * edges[-1])]

    # Evaluate (f*f)(t) at each breakpoint
    a = edges[:-1]  # left edges of bins
    b = edges[1:]   # right edges of bins

    peak = -np.inf
    peak_t = None

    batch_size = 500
    for start in range(0, len(bp), batch_size):
        end = min(start + batch_size, len(bp))
        t_batch = bp[start:end]

        # For each (i,j) bin pair, overlap of [a_i, b_i] and [t - b_j, t - a_j]
        # is max(0, min(b_i, t - a_j) - max(a_i, t - b_j))
        conv = np.zeros(len(t_batch))
        for i in range(N):
            for j in range(N):
                lo = np.maximum(a[i], t_batch - b[j])
                hi = np.minimum(b[i], t_batch - a[j])
                overlap = np.maximum(0.0, hi - lo)
                conv += heights[i] * heights[j] * overlap

        idx = np.argmax(conv)
        if conv[idx] > peak:
            peak = conv[idx]
            peak_t = t_batch[idx]

    return float(peak), float(peak_t)


print("Verification method 1 (exact breakpoint) defined.")

In [ ]:
# === Verification Method 2: Direct numerical integration on fine grid ===

def verify_direct_integration(edges, heights, n_t=10001):
    """
    Evaluate (f*f)(t) = integral f(x)f(t-x)dx on a fine grid of t values.
    Uses midpoint rule with small dx for the inner integral.
    """
    edges = np.asarray(edges, dtype=np.float64)
    heights = np.asarray(heights, dtype=np.float64)

    support_lo, support_hi = edges[0], edges[-1]
    t_lo = 2 * support_lo
    t_hi = 2 * support_hi

    # Fine grid for x (inner integral)
    n_x = 10000
    dx = (support_hi - support_lo) / n_x
    x_grid = support_lo + (np.arange(n_x) + 0.5) * dx

    # Evaluate f on x_grid
    bin_idx = np.searchsorted(edges, x_grid, side='right') - 1
    bin_idx = np.clip(bin_idx, 0, len(heights) - 1)
    in_support = (x_grid >= support_lo) & (x_grid < support_hi)
    f_x = np.where(in_support, heights[bin_idx], 0.0)

    # Grid for t
    t_grid = np.linspace(t_lo, t_hi, n_t)

    peak = -np.inf
    peak_t = None

    for t in t_grid:
        # f(t - x) evaluated at each x_grid point
        tmx = t - x_grid
        idx2 = np.searchsorted(edges, tmx, side='right') - 1
        idx2 = np.clip(idx2, 0, len(heights) - 1)
        in_supp2 = (tmx >= support_lo) & (tmx < support_hi)
        f_tmx = np.where(in_supp2, heights[idx2], 0.0)

        val = np.sum(f_x * f_tmx) * dx
        if val > peak:
            peak = val
            peak_t = t

    return float(peak), float(peak_t)


print("Verification method 2 (direct integration) defined.")

In [ ]:
# === Verification Method 3: FFT convolution ===

def verify_fft(edges, heights, n_grid=65536):
    """
    FFT-based autoconvolution on a very fine grid.
    """
    edges = np.asarray(edges, dtype=np.float64)
    heights = np.asarray(heights, dtype=np.float64)

    support_lo, support_hi = edges[0], edges[-1]
    dx = (support_hi - support_lo) / n_grid
    x = support_lo + np.arange(n_grid) * dx

    # Evaluate f on grid
    bin_idx = np.searchsorted(edges, x, side='right') - 1
    bin_idx = np.clip(bin_idx, 0, len(heights) - 1)
    in_support = (x >= support_lo) & (x < support_hi)
    f_vals = np.where(in_support, heights[bin_idx], 0.0)

    # Zero-pad and FFT convolve
    M = 2 * n_grid
    f_padded = np.zeros(M)
    f_padded[:n_grid] = f_vals
    F = np.fft.rfft(f_padded)
    conv = np.fft.irfft(F * F) * dx

    # Physical t coordinates
    t_vals = 2 * support_lo + np.arange(M) * dx
    mask = (t_vals >= 2 * support_lo) & (t_vals <= 2 * support_hi)

    peak = float(np.max(conv[mask]))
    peak_idx = np.argmax(conv[mask])
    peak_t = float(t_vals[mask][peak_idx])

    return peak, peak_t


print("Verification method 3 (FFT) defined.")

In [ ]:
# === Verification Method 4: Discrete simplex objective ===

def verify_discrete(simplex_weights, P):
    """
    Compute max_k 2P * sum_{i+j=k} x_i x_j directly from simplex weights.
    This is the discretized objective that the optimizer actually minimizes.
    """
    x = np.asarray(simplex_weights, dtype=np.float64)
    c = np.convolve(x, x) * (2.0 * P)
    peak = float(np.max(c))
    k_star = int(np.argmax(c))
    return peak, k_star


print("Verification method 4 (discrete) defined.")

In [ ]:
# === Feasibility checks ===

def check_feasibility(edges, heights, simplex_weights):
    """Check f >= 0 and integral(f) = 1."""
    edges = np.asarray(edges, dtype=np.float64)
    heights = np.asarray(heights, dtype=np.float64)
    x = np.asarray(simplex_weights, dtype=np.float64)

    issues = []

    # Non-negativity
    min_h = float(np.min(heights))
    if min_h < -1e-10:
        issues.append(f"FAIL: f has negative values (min = {min_h:.2e})")

    # Integral = 1
    widths = np.diff(edges)
    integral = float(np.sum(heights * widths))
    if abs(integral - 1.0) > 1e-8:
        issues.append(f"FAIL: integral(f) = {integral:.10f} != 1")

    # Simplex weights sum to 1
    x_sum = float(np.sum(x))
    if abs(x_sum - 1.0) > 1e-8:
        issues.append(f"FAIL: simplex weights sum = {x_sum:.10f} != 1")

    # Simplex weights non-negative
    min_x = float(np.min(x))
    if min_x < -1e-10:
        issues.append(f"FAIL: simplex weights have negative values (min = {min_x:.2e})")

    return issues


print("Feasibility checker defined.")

In [ ]:
# === Run full verification on all solutions ===

print(f"{'Label':<20} | {'Claimed':>10} | {'Exact BP':>10} | {'Direct':>10} | {'FFT':>10} | {'Discrete':>10} | {'Status'}")
print('-' * 105)

verification_results = {}

for label in sorted(all_solutions.keys()):
    s = all_solutions[label]
    P = s['P']
    x = np.array(s['simplex_weights'])

    # Compute edges and heights if not provided
    if 'edges' in s:
        edges = np.array(s['edges'])
        heights = np.array(s['heights'])
    else:
        # Reconstruct from simplex weights (uniform grid on [-1/4, 1/4])
        edges = np.linspace(-0.25, 0.25, P + 1)
        bin_width = 0.5 / P
        heights = x / bin_width  # heights such that integral = sum(x) = 1

    claimed = s.get('claimed_exact_peak', s.get('exact_peak'))

    # Feasibility
    issues = check_feasibility(edges, heights, x)

    # Method 1: exact breakpoint
    exact_peak, exact_t = verify_exact_breakpoint(edges, heights)

    # Method 2: direct integration (skip for large P to save time)
    if P <= 100:
        direct_peak, direct_t = verify_direct_integration(edges, heights, n_t=5001)
    else:
        direct_peak = float('nan')

    # Method 3: FFT
    fft_peak, fft_t = verify_fft(edges, heights, n_grid=65536)

    # Method 4: discrete
    disc_peak, disc_k = verify_discrete(x, P)

    # Check consistency
    tol = 1e-4
    if abs(exact_peak - claimed) > tol:
        issues.append(f"MISMATCH: claimed {claimed:.6f} vs exact {exact_peak:.6f}")
    if not np.isnan(direct_peak) and abs(direct_peak - exact_peak) > 0.01:
        issues.append(f"DIRECT DISAGREES: {direct_peak:.6f} vs exact {exact_peak:.6f}")
    if abs(fft_peak - exact_peak) > 0.01:
        issues.append(f"FFT DISAGREES: {fft_peak:.6f} vs exact {exact_peak:.6f}")

    status = "PASS" if len(issues) == 0 else "FAIL"

    direct_s = f"{direct_peak:>10.6f}" if not np.isnan(direct_peak) else f"{'skip':>10}"
    print(f"{label:<20} | {claimed:>10.6f} | {exact_peak:>10.6f} | {direct_s} | {fft_peak:>10.6f} | {disc_peak:>10.6f} | {status}")

    if issues:
        for issue in issues:
            print(f"  >> {issue}")

    verification_results[label] = {
        'claimed': claimed,
        'exact_breakpoint': exact_peak,
        'direct_integration': direct_peak,
        'fft': fft_peak,
        'discrete': disc_peak,
        'status': status,
        'issues': issues,
    }

In [ ]:
# === Summary: best verified upper bound ===

passed = {k: v for k, v in verification_results.items() if v['status'] == 'PASS'}
failed = {k: v for k, v in verification_results.items() if v['status'] == 'FAIL'}

print(f"Passed: {len(passed)} / {len(verification_results)}")
print(f"Failed: {len(failed)} / {len(verification_results)}")

if failed:
    print("\nFailed solutions:")
    for k, v in failed.items():
        print(f"  {k}: {v['issues']}")

if passed:
    best_label = min(passed, key=lambda k: passed[k]['exact_breakpoint'])
    best = passed[best_label]
    print(f"\n{'='*60}")
    print(f"BEST VERIFIED UPPER BOUND")
    print(f"{'='*60}")
    print(f"Solution:           {best_label}")
    print(f"Claimed value:      {best['claimed']:.6f}")
    print(f"Exact breakpoint:   {best['exact_breakpoint']:.6f}")
    print(f"FFT check:          {best['fft']:.6f}")
    print(f"Discrete check:     {best['discrete']:.6f}")
    s = all_solutions[best_label]
    print(f"P (bins):           {s['P']}")
    print(f"\nThis certifies: C_{{1a}} <= {best['exact_breakpoint']:.6f}")
    print(f"Best known:         C_{{1a}} <= 1.5029")
    print(f"Gap to best known:  {best['exact_breakpoint'] - 1.5029:+.6f}")